# Improving the NLPD using the confusion matrix

## Importing modules and loading data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
layerXtest = pd.read_csv('Weights2 - 6th  layer/Uk_weights2_X_test.csv',header=None).to_numpy()
layerYtest = pd.read_csv('Weights2 - 6th  layer/Uk_weights2_Y_test.csv',header=None).to_numpy()
layerXtrain = pd.read_csv('Weights2 - 6th  layer/Uk_weights2_X_validation.csv',header=None).to_numpy()
layerYtrain = pd.read_csv('Weights2 - 6th  layer/Uk_weights2_Y_validation.csv',header=None).to_numpy()
smXtest = pd.read_csv('Weights2 - Softmax/Uk_weights2_X_test.csv',header=None).to_numpy()
smYtest = pd.read_csv('Weights2 - Softmax/Uk_weights2_Y_test.csv',header=None).to_numpy()
smXtrain = pd.read_csv('Weights2 - Softmax/Uk_weights2_X_validation.csv',header=None).to_numpy()
smYtrain = pd.read_csv('Weights2 - Softmax/Uk_weights2_Y_validation.csv',header=None).to_numpy()

#we'll include the training data too...
smXtrain2 = pd.read_csv('Weights2 - Softmax/Uk_weights2_X_train.csv',header=None).to_numpy()
smYtrain2 = pd.read_csv('Weights2 - Softmax/Uk_weights2_Y_train.csv',header=None).to_numpy()
smXtrain = np.r_[smXtrain,smXtrain2]
smYtrain = np.r_[smYtrain,smYtrain2]

In [3]:
#If we want to just use the original test data and split it into training + test.
orgtest_smXtrain = smXtest[::2,:]
orgtest_smXtest = smXtest[1::2,:]
orgtest_smYtrain = smYtest[::2,:]
orgtest_smYtest = smYtest[1::2,:]

In [4]:
def NLPD(actual,predicted):
    """(normalised) Negative Log Predictive Density
    Definition of Negative Log Predictive Density (NLPD):
    $$L = -\frac{1}{n} \sum_{i=1}^n \log p(y_i=t_i|\mathbf{x}_i)$$
    See http://mlg.eng.cam.ac.uk/pub/pdf/QuiRasSinetal06.pdf, page 13.
    "This loss penalizes both over and under-confident predictions."
    but
    "The NLPD loss favours conservative models, that is models that tend to be under-confident
    rather than over-confident. This is illustrated in Fig. 7, and canbe deduced from the fact that
    logarithms are being used. An interesting way ofusing the NLPD is to give it relative to the NLPD
    of a predictor that ignoresthe inputs and always predicts the same Gaussian predictive distribution,
    withmean and variance the empirical mean and variance of the training data. Thisrelative NLPD
    translates into a gain of information with respect to the simpleGaussian predictor described."
    
    actual = 2d array of indices (Nx1), values are from 0 to D-1.
    predicted = 2d array of probabilities (NxD) sum to one along rows.
    """
    assert np.max(np.abs(np.sum(predicted,1)-1))<1e-5 #confirmed every row roughly adds up to one
    assert 0<=np.max(predicted)<=1
    return -np.sum(np.log(np.take_along_axis(predicted,actual.astype(int),axis=1)))

def MNLPD(actual,predicted):
    """Mean of the NLPD"""
    assert np.max(np.abs(np.sum(predicted,1)-1))<1e-5 #confirmed every row roughly adds up to one
    assert 0<=np.max(predicted)<=1
    return -np.mean(np.log(np.take_along_axis(predicted,actual.astype(int),axis=1)))
def GMNLPD(actual,predicted):
    """Grouped Mean NLPD
    This finds the mean of the MNLPDs for all the separate actual groups.
    It means that if one large group is well classified it won't swamp the results from
    the smaller groups"""
    assert np.max(np.abs(np.sum(predicted,1)-1))<1e-5 #confirmed every row roughly adds up to one
    assert 0<=np.max(predicted)<=1
    res = []
    for i in np.unique(actual):
        res.append(MNLPD(actual[actual[:,0]==i],predicted[actual[:,0]==i,:]))
    return np.mean(res)


def build_conf(smXtrain,smYtrain):
    preds = np.argmax(smXtrain,1)
    conf = np.zeros([19,19])
    for p,a in zip(preds,smYtrain[:,0].astype(int)):
        conf[p,a]+=1.0
    conf+=0.01
    conf = (conf.T/np.sum(conf,1)).T
    return conf

def getprobs(conf,smXtest):
    newprobs = np.zeros_like(smXtest)
    preds = np.argmax(smXtest,1)
    for i,p in enumerate(preds):
        newprobs[i,:] = conf[p,:]
    return newprobs

def build_prob_conf(smXtrain,smYtrain,normalise_by_predclass=False):
    """
    set normalise_by_predclass true for best GMNLPD score.
    """
    conf = np.zeros([19,19])
    for x,a in zip(smXtrain,smYtrain[:,0].astype(int)):
        conf[:,a]+=x

    #disable this line for NLPD, enable for GMNLPD.
    if normalise_by_predclass:
        conf = (conf/np.sum(conf,0))
    conf = (conf.T/np.sum(conf,1)).T
    conf+=np.eye(19)*1
    conf = (conf.T/np.sum(conf,1)).T
    return conf

def regularise(mat):
    tempX = mat+0.01
    tempX=(tempX.T/np.sum(tempX,1)).T
    return tempX

# Results

Below we look at the NLPD and also the Grouped Mean NLPD 'GMNLPD' this latter measure weights the different species classes equally (and finds the mean per sample).

## CNN Softmax solutions

In [5]:
print("Deep Neural network")
print("(using original training data) NLPD: %8.1f, GMNLPD: %6.3f" % (NLPD(smYtest,smXtest),GMNLPD(smYtest,smXtest)))
print("(using original test data)     NLPD: %8.1f, GMNLPD: %6.3f" % (NLPD(orgtest_smYtest,orgtest_smXtest),GMNLPD(orgtest_smYtest,orgtest_smXtest)))
print("\nTrying to regularise the CNN softmax solution to see if it helps (as we regularise our EUE method, so it's fair to try it here too).\n")

print("Regularised Deep Neural network")
tempX = regularise(smXtest)
print("(using original training data) NLPD: %8.1f, GMNLPD: %6.3f" % (NLPD(smYtest,tempX),GMNLPD(smYtest,tempX)))
tempX = regularise(orgtest_smXtest)
print("(using original test data)     NLPD: %8.1f, GMNLPD: %6.3f" % (NLPD(orgtest_smYtest,tempX),GMNLPD(orgtest_smYtest,tempX)))

Deep Neural network
(using original training data) NLPD:   5655.7, GMNLPD:  1.566
(using original test data)     NLPD:   2791.4, GMNLPD:  1.361

Trying to regularise the CNN softmax solution to see if it helps (as we regularise our EUE method, so it's fair to try it here too).

Regularised Deep Neural network
(using original training data) NLPD:   5764.6, GMNLPD:  1.302
(using original test data)     NLPD:   2877.5, GMNLPD:  1.201


Regularising improves the GMNLPD but not the NLPD.

## Empirical Uncertainty Estimation

Using just the confusion matrix:

In [8]:
conf = build_conf(smXtrain,smYtrain)
newprobs = getprobs(conf,smXtest)
print("(using original training data) NLPD: %8.1f, GMNLPD: %6.3f" % (NLPD(smYtest,newprobs),GMNLPD(smYtest,newprobs)))
conf = build_conf(orgtest_smXtrain,orgtest_smYtrain)
newprobs = getprobs(conf,orgtest_smXtest)
print("(using origial test data) NLPD:      %8.1f, GMNLPD: %6.3f" % (NLPD(orgtest_smYtest,newprobs),GMNLPD(orgtest_smYtest,newprobs)))

(using original training data) NLPD:   5415.5, GMNLPD:  2.333
(using origial test data) NLPD:        2744.8, GMNLPD:  2.275


Using the probabilities to build a sort of "probability confusion matrix" (this might have a name)...

In [12]:
conf = build_prob_conf(smXtrain,smYtrain)
newprobs = smXtest@conf
print("(using original training data) NLPD: %8.1f, GMNLPD: %6.3f" % (NLPD(smYtest,newprobs),GMNLPD(smYtest,newprobs)))
conf = build_prob_conf(orgtest_smXtrain,orgtest_smYtrain)
newprobs = orgtest_smXtest@conf
print("(using origial test data) NLPD:      %8.1f, GMNLPD: %6.3f" % (NLPD(orgtest_smYtest,newprobs),GMNLPD(orgtest_smYtest,newprobs)))


(using original training data) NLPD:   4473.8, GMNLPD:  1.457
(using origial test data) NLPD:        2228.1, GMNLPD:  1.313


We can minimise the NLPD by simply transforming by the confusion probability matrix.
The GMNLPD is minimised by normalising by the number of observations (this obviously harms the
overall NLPD but the GMNLPD is interested in the average over each actual species).

In [14]:
print("Normalising the probability confusion matrix")
conf = build_prob_conf(smXtrain,smYtrain,normalise_by_predclass=True)
newprobs = smXtest@conf
print("(using original training data) NLPD: %8.1f, GMNLPD: %6.3f" % (NLPD(smYtest,newprobs),GMNLPD(smYtest,newprobs)))
conf = build_prob_conf(orgtest_smXtrain,orgtest_smYtrain,normalise_by_predclass=True)
newprobs = orgtest_smXtest@conf
print("(using origial test data) NLPD:      %8.1f, GMNLPD: %6.3f" % (NLPD(orgtest_smYtest,newprobs),GMNLPD(orgtest_smYtest,newprobs)))

Normalising the probability confusion matrix
(using original training data) NLPD:   5321.5, GMNLPD:  1.168
(using origial test data) NLPD:        2857.1, GMNLPD:  1.092


So if you want the best NLPD don't normalise by number in each class - in both datasets this results in a 20\% reduction in the NLPD. If you are concerned about the NLPD with equal weighting by class then normalise, this approach led to a 10\% reduction in the GMNLPD in both datasets.

In [18]:
4474/5655,2228/2791,1.092/1.201,1.168/1.302

(0.791158267020336, 0.7982801863131495, 0.9092422980849293, 0.8970814132104454)